목표 : 자동 매핑 안된 개체들 수기로 네이버 프로필 주소 수집하기

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import datetime
from difflib import SequenceMatcher
import pymysql
from sqlalchemy import create_engine
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

In [ ]:
def db_connection(host_name='ds'):
    host_url = "db.ds.mycelebs.com"
    user_nm = 
    passwd = 
    port_num = 3306
    db_name = "star_ko"
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8',
                           db=db_name, cursorclass=pymysql.cursors.DictCursor)
    # cursorclass=pymysql.cursors.DictCursor 추가 -> 데이터프레임으로 다루기 쉽게 딕셔너리 형태로 결과 반환해줌, cursor는 튜플형태
    # db=db,
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    return conn, engine

conn, engine = db_connection()

In [ ]:
qry = 'SELECT * FROM star_ko_profile_url WHERE naver_url=\'0\''
data = pd.read_sql(qry, conn)

data.head()

In [ ]:
len(data)

In [ ]:
cursor = conn.cursor(pymysql.cursors.DictCursor)

number = int(input("url 수집할 개체 수 입력 :"))
ex = data[0:number]

chrome_options = webdriver.ChromeOptions()
naver_drv = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)
profile = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

for i in range(len(ex)-1) :
    profile.execute_script('window.open("about:blank", "_blank");')
    
tabs = profile.window_handles

for i in range(len(ex)) :
    profile.switch_to_window(tabs[i])
    profile.get(f'https://www.mycelebs.ai/celeb/{int(ex["series_id"].iloc[i])}')
    try :
        profile.find_element_by_xpath('//*[@id="closedOneWeek"]').click()
    except :
        pass

for i, row in ex.iterrows() :
    series_id = row['series_id']
    name = row['name']
    
    
    naver_drv.get(f'https://search.naver.com/search.naver?where=m&sm=mtb_etc&mra=bjky&pkid=1&qvt=0&query={name}')
    
    print(name, ' - ', series_id)
    naver_url = input("url을 입력하세요(없으면 입력 X): ")
    
    if naver_url == '' :
        qry = f'update star_ko_profile_url set naver_url=\'x\', update_date=NOW() where series_id={series_id};'
        cursor.execute(qry)
        conn.commit()
    else :
        naver_id = re.findall('&os=(\d+)', naver_url)    
        qry = f"update star_ko_profile_url set naver_url='{naver_url}', naver_people_id='{naver_id[0]}', update_date=NOW() where series_id={series_id};"
        cursor.execute(qry)
        conn.commit()
    
conn.close()
naver_drv.close()
profile.close()